In [2]:
import torch
import numpy as np
from sklearn.preprocessing import StandardScaler




import xlrd
import pandas as pd

book = xlrd.open_workbook('dataset for standardisation.xlsx')
sheet = book.sheet_by_name('Sheet1')
data_unreplicated = [[sheet.cell_value(r, c)
         for c in range(1,9)] for r in range(2,74)]
scaler = StandardScaler()
scaler.fit(data_unreplicated)

print(scaler.mean_, scaler.var_)

book = xlrd.open_workbook('prepareddata.xlsx')
sheet = book.sheet_by_name('Sheet1')
replicated_data = [[sheet.cell_value(r, c)
         for c in range(0,8)] for r in range(0,7272)]

standardised_data = scaler.transform(replicated_data)
np.random.shuffle(standardised_data)
array1 = standardised_data[:, 0:5]
array2 = standardised_data[:, 5:8]
x = torch.tensor(array1)
y = torch.tensor(array2)

print(x)
print(y)


[1.17675597e+00 2.50709764e+03 2.03210694e+00 3.30000000e+02
 3.00000000e-01 1.51537037e-02 4.70249769e+01 2.63078704e-02] [5.28799796e-01 1.01306701e+07 1.74931795e+00 2.70000000e+04
 4.00000000e-02 2.61359971e-05 5.06386030e+03 2.10257371e-04]
tensor([[ 2.0648,  3.4563,  1.7584,  0.9129,  1.0000],
        [-1.5128, -0.5423, -1.5364,  1.6432, -1.0000],
        [ 0.8102, -0.3550,  1.0736, -0.1826, -1.0000],
        ...,
        [ 1.2838, -0.2114,  1.2788, -0.1826, -1.0000],
        [-0.6283, -0.6897, -0.6911, -0.1826,  1.0000],
        [ 0.4352, -0.1845,  0.3689, -1.0954, -1.0000]], dtype=torch.float64)
tensor([[-2.4590,  1.1765, -0.1819],
        [ 1.5175, -0.7490,  1.6767],
        [ 0.5515, -0.3789, -1.2194],
        ...,
        [-1.7414, -0.3063, -3.1062],
        [ 1.5242, -0.7348, -0.1501],
        [-3.1545, -0.2944, -2.5301]], dtype=torch.float64)


In [3]:
import torch.nn as nn
class Net(nn.Module):
    def __init__(self, D_in, H1, H2, D_out):
 
        super(Net, self).__init__()
        self.linear1 = nn.Linear(D_in, H1) 
        self.linear2 = nn.Linear(H1, H2)
        self.linear3 = nn.Linear(H2, D_out)
  
    def forward(self, x):
    
        x = torch.sigmoid(self.linear1(x))
        x = torch.sigmoid(self.linear2(x))
        x = self.linear3(x)
        return x



In [4]:
# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H1, H2, D_out = 50, 5, 20, 12, 3
model = Net(D_in, H1, H2, D_out)
loss_fn = torch.nn.MSELoss(reduction='mean')

100%|███████████████████████████████████████████████████████████████████████████████| 146/146 [00:00<00:00, 508.75it/s]


0 1.4624454975128174


100%|███████████████████████████████████████████████████████████████████████████████| 146/146 [00:00<00:00, 920.46it/s]


1 1.1826165914535522


100%|███████████████████████████████████████████████████████████████████████████████| 146/146 [00:00<00:00, 920.95it/s]


...

18 0.5009312629699707


100%|███████████████████████████████████████████████████████████████████████████████| 146/146 [00:00<00:00, 982.49it/s]


19 0.48324331641197205


100%|███████████████████████████████████████████████████████████████████████████████| 146/146 [00:00<00:00, 939.45it/s]


20 0.4674999415874481
tensor([[-0.6218, -0.7024,  0.3110],
        [-0.9054,  0.9122, -0.2337],
        [-0.4277,  0.9579, -0.4600],
        [-1.1800, -0.6826,  0.4420],
        [-0.6185, -0.4161, -0.4296],
        [ 1.5696, -0.7599,  1.7698],
        [ 1.4593, -0.7337, -0.0959],
        [ 0.7082, -0.4049,  0.3363],
        [ 2.0233, -0.7023,  0.7455],
        [-0.8242, -0.3994, -2.3401],
        [ 1.0950, -0.7487,  1.6275],
        [ 1.2591,  1.5909, -0.8053],
        [-0.6527, -0.9746,  0.0714],
        [ 0.3676, -0.7843,  0.3007],
        [-0.8101, -0.3641,  1.4566],
        [-0.5074,  1.0160, -0.9332],
        [-1.5097,  1.8185, -1.4950],
        [-2.7136, -0.2823, -3.0610],
        [-0.5827,  0.7602, -0.6508],
        [-1.7414, -0.3063, -3.1062],
        [ 1.5242, -0.7348, -0.1501],
        [-3.1545, -0.2944, -2.5301]], dtype=torch.float64)
tensor([[ 0.0189, -0.7248,  0.4013],
        [-0.6114,  1.4631, -0.5895],
        [-0.1259,  1.3603, -0.2834],
        [-0.2447, -0.7362,  0.1

In [6]:
torch.save(model.state_dict(), ("C:/Users/Gabriel/Documents/Python/ANN2.pt"))                
                                

In [30]:
print(data2)

[[0.01783333333333333, 24.988333333333344, 0.050833333333333335], [0.023416666666666665, -32.44333333333335, 0.04166666666666668], [0.02313333333333334, -9.365833333333322, 0.041666666666666664], [0.020691666666666674, 32.817499999999995, 0.034999999999999976], [0.019733333333333325, -26.63583333333334, 0.03500000000000003], [0.015975, 132.69500000000005, 0.02749999999999997], [0.01776666666666667, 242.3891666666666, 0.027500000000000007], [0.012699999999999989, 146.2941666666667, 0.029166666666666674], [0.013891666666666672, 193.01666666666657, 0.01416666666666666], [0.01153333333333332, 129.1691666666667, 0.025000000000000022], [0.009850000000000025, 118.19666666666672, 0.01833333333333335], [0.010924999999999999, 140.48833333333323, 0.00916666666666662]]
